# BioFit

This JupyterLab book can be copied and used for each different use case that you may have for the determination of empirical kenetic parameters for saturation and inhibition growth kinetics. As incrimental changes are made to various parts to the code it is recommended that you operate a git pull before obtaining a set of parameters for a collective model

#### BioFit domain

Before starting what is the domain where you have stored biofit? Enter its location here or if is in the current working directory then can leave empty

In [6]:
BioFit_path = 'Insert path'

import sys
sys.path.append(BioFit_path)

import os
BioFit_path = os.getcwd()

## Data input

Input your data here for each use case of interest. 
You may want to make note of which use case this is here:

What data set are you using?

In [7]:
####Data sets to imported


## Data input selection

You may have multiple data sets, some of which you may want to select but not others. For this case specify the number of data sets you are using and which

In [8]:
#Do you only want to consider only 1 data set? - if so set this to 1
no_datasets = 1
#Input lists of data sets that you want to use
if no_datasets == 1:
    ###Data sets to use
    xdata_sets = [None]
    #ydata
    ydata_sets = [None]
    #yerr data sets
    yerr_sets = [None]
else:
    ###Data sets to use
    xdata_sets = [None]
    #ydata
    ydata_sets = [None]
    #yerr data sets
    yerr_sets = [None]

### Data Scaling

It may be that you are considering multiple data sets which have variation in the y-axis due to inter-cell source variablilty. If you want to combine multiple data sets and extract the features from all then you may want to scale. 

In [9]:
#If you are using multiple data sets, do you want to scale them along the y axis to account for variation which occours due to maximum production rates
scale = 'Yes'

## Fitting Options

Has your data set got error associated with each point? If yes then set error to Yes else No

In [10]:
error = 'Yes'

#### Estimated Variables

According to menton kinetics it is possible to estimate certain growth kinetics from the data set. An example of this would include the maximum growth rate or production rate. From this you can detertmine the Ks value which is defined as the magnitude of the stimulant at half the maximal growth rate or production rate. It may therefore be desirible to estimate variables.

If you believe that your data set encompasses a wide enough range to estimate variables then enter yes, else leave as no. - note that it may be necessary to estimate variables if the fitted variables are outside the range of reason, therefore be sure to check final values make sense.

In [11]:
#Do you want to use estimated variables? - If you are getting a bad fit try fitting without using estimated variables
#If you are not using the estimated variables, check the fitted parameters to make sure that they make sense
Estimated_var = 'No'

#### Type of fit

From the data what kind of profile are you expecting?

In [12]:
#When you look at the raw data, does it roughly fit a logistic, exponential or inverse log profile?
#select from: log_growth, exp_growth or inverse_log
profile = 'log_growth'

#Do you have basal production?
basal = 'Yes'

#### More options

In [13]:
#What is your minimum acceptable R^2 value to accept a fitted model
min_r2 = 0.8
#What is the smallest x-axis value you want to plot
xmin_plot = 1e-7
#what value do you replace 0s with in x data
x0_replace = xmin_plot
#which models do you want to try fit and plot 
models = ['Menten','Luong','Andrews','Aiba','Moser','Webb','Yano','Haldane','Edward','Han-Levenspiel','Inverse_Monod','exp_growth']
##Do you want to print out fitting values?
fit_print = 'No'

## Plotting Options

#### Axis lables

Input axis labels for resulting figures, be sure to put the right units

In [14]:
#X-axis label
xlabel=r'x-axis label'
#Y-axis label
ylabel=r'y-axis label'
#Do you want to plot a figure legend?
legend_plot = 'Yes'
#How many points do you want between each xaxis data point when plotting funtions
x_plotno = 30
#Do you want to calculate function values only within that of the experimental data?
max_check = 'Yes'
#If you want to calculate to another value what is it?
max_x = 200

## Saving options

In [15]:
#What directory do you want to save the figure to?
root =  'Where do you want everything to save insert the path here'
#What do you want to call the figure which you are saving?
figure_name = 'What is your figure file name'
#What do you want to call the data file which you are saving?
data_name = 'What is your data output file name'

## Import libraries and functions

In [16]:
from Biofit_functions import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import importlib
from BioFit_options import *

ModuleNotFoundError: No module named 'Biofit_functions'

## Estimation of variables

If combining multiple data sets it may be there there are multiple data points associated with each x-axis point. To avoid errors occouring while fitting models to data need to therefore combine and calculate averages for these points

In [ ]:
xdata,ydata,yerr = comb_set(no_datasets,scale,xdata_sets,ydata_sets,yerr_sets,x0_replace,error)
print(xdata,ydata,yerr)

According to menten kinetics it is possible to estimate kinetic parameters by taking the largest rate as a maximum expression or growth rate. The Ks parameter is then half the magnitude of the sitimulus at associated with this maximal value. We can therefore estimate these values if the data set provided is at a high enough resolution across the range of interest

In [ ]:
#ad_bounds = esti_var(Estimated_var,ydata,xdata)[2]
Smin,bounds,ad_bounds = esti_var(Estimated_var,ydata,xdata)

print(bounds)
print(Smin)

When plotting we must calculate y-axis values associated with each x-axis point after we have determined the kinetic parameters associated with each model in order to plot them. To ensure a constant resolution of this plot we determine which x-axis values we should consider when calculating y-axis points.

In [ ]:
xdata_plot = xsteps(xdata,x_plotno,xmin_plot,max_check,max_x)
#print(xdata_plot)

## Fitting and Plotting of models

In [ ]:
#Create dataframe into which variables are to be saved
ep_df = pd.DataFrame()

#Create figure to plot data and fitted funcitons onto
fig, ax = plt.subplots()

#Plot raw experimental data
if error == 'Yes':
    ax.errorbar(xdata, ydata, yerr=yerr, fmt='o')
else:
    ax.scatter(xdata, ydata, marker='o')

#Want to dynamically import function associated with each model of interest. To do this will import functions from bio_fit models package
#Initially want to to programatically make a string of function of interest
#Check if basal production
if basal == 'Yes':
    #In the case of inverse_log while it may be that there is basal production only consider on set of models
    if profile == 'inverse_log':
        full_module_name = 'biofit_models.' + profile + '.' +'models'
    #Select models of those with basal production
    else:
        full_module_name = 'biofit_models.' + profile + '.' + 'basal.' +'models'
    #Select models which have no basal production and are not inverse log
else:
    full_module_name = 'biofit_models.' + profile + '.' +'models'
#Having made a string of the module to be imported then import module
mymodule = importlib.import_module(full_module_name)

#Having imported module containing models want to check if the specified models exist within the module
for model in models:
    if model not in dir(mymodule):       
        pass
    else:
        #select the model from the module if the 
        function = getattr(mymodule, model)
        
        ####Having retrieved the model then proceed to fit###
        abounds = ad_bounds[model]
        # print(bounds)
        bbounds = bounds[model]
        
#         print(abounds)
#         print(bbounds)
        pop, pcov = fitting(basal=basal,function=function,model=model,xdata=xdata,ydata=ydata,error=error,sigma=yerr,bounds=bbounds,abounds=abounds)
        
        #Having fit the function want to save associated fitting data to dataframe
        #Create list of variables associated with function
        var_list = list(function.__code__.co_varnames)
        var_list.remove('S')
        #append fitted values and statistics to global dataframe
        ep_df,r_squared,max_function_y = fit_figures(basal,xdata,ydata,xdata_plot,var_list,pop,model,function,ep_df)
        #print(max_function_y)
        #Print out statement of fitted values and associated statistics for easy quick check
        if fit_print == 'Yes':
            print(model+str(pop))
            print(model+'r_squared='+str(r_squared))
            
        
        #Plot from fitted model figures
        #check to make sure worth plotting
        if r_squared < min_r2:
            pass
        else:
            ax.plot(xdata_plot, function(xdata_plot, *pop), linestyle=model_style[model], color=model_colour[model],label=model)
#Set labels of plot
ax.set(xlabel=xlabel, ylabel=ylabel)
#plotting legend 
if legend_plot == 'Yes':
    plt.legend()

#Set axis bounds of plot
#Axis limits in figure
#X-axis
xmin = 0
xmax = max(xdata)*1.2
#Y-axis
ymax = 0
ymin = max(ydata)*1.2
#setting bounds
axes = plt.gca()
#axes.set_xlim([xmin,xmax])
#axes.set_ylim([ymax,ymin])
#Make figure tight
fig.tight_layout()
plt.show()
#Specify figure save location
figure_loc = root+'figures/'
#make sure figure save location exists
checkdir(figure_loc)
#Save figure out
fig.savefig(figure_loc+figure_name, dpi=300)

##Saving Data out
#Save out global fitted function value dataframe
#Move model column to be first column for ease of reading
ep_df.insert(0, 'Model', ep_df.pop('model'))
#Reorder models based on R^2 value
ep_df = ep_df.sort_values(by=['R Squared'],ascending=False)
#Reset pandas dataframe index
ep_df = ep_df.reset_index(drop=True)
#Specify data save location
data_loc = root+'data/'
#make sure data file location exists
checkdir(data_loc)
#Save dataframe out 
ep_df.to_csv(data_loc+data_name+'.csv')
#Print dataframe for ease of quick check
print(ep_df)